# Read the .set files using mne then use MNF to do dimension reduction
* This notebook will use MNE to read the .set raw data and then use MNF to do feature selection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
import os.path as op
from os import listdir
from os.path import isfile, join


In [ ]:
raw=mne.io.read_raw_eeglab('C:\\USB Drive\\NewEEG-200s\\SA20140205\\20140205_1230.set',preload=True)

In [ ]:
raw.plot()

In [ ]:
epochs=mne.read_epochs_eeglab('C:\\USB Drive\\raw_data\\SA20140205\\20140205_1230.set', uint16_codec='utf-8')

In [ ]:
rawfilter=raw.filter(3,10)

In [ ]:
rawfilter.plot()

In [ ]:
raw.ch_names

In [ ]:
mne.Epochs(raw,mne.read_events(raw))

In [ ]:
mne.find_events(raw)

In [ ]:
mne.read_events(raw)

In [ ]:
raw.pick_types(meg=False, eeg=True, eog=True)

In [ ]:
events = mne.find_events(raw, min_duration=1/raw.info['sfreq'], shortest_event=1)

In [ ]:
raw.crop(0, 20)

In [ ]:
raw.plot(n_channels= 20, duration= 10, start=10)

In [ ]:
rawfilter=raw.filter(10,100, fir_design='firwin')

In [ ]:
rawfilter.plot()

In [ ]:
events = mne.find_events(raw, min_duration=1/raw.info['sfreq'], shortest_event=1)

In [ ]:
# event_id : this dictionary is created from information from Dave's dissertation page 27
concepts={1:'BABY',2:'BELL',3:'BIRD',4:'BURP',5:'DOG',6:'DRUM',
              7:'KNOCK',8:'LAUGH',9:'PHONE',10:'TRAIN',11:'WATER'}

event_id = {'trail_start':  31, 
            'left_button':  131, 
            'right_button': 132,
            'congruent':    133,
            'incongruent':  134,
            'correct':      129,
            'incorrect':    130,
            # targets
            't_baby' :        135,
            't_bell':         136,
            't_bird':         137,
            't_burp':         138,
            't_dog':          139,
            't_drum':         140,
            't_knock':        141,
            't_laugh':        142,
            't_phone':        143,
            't_train':        144,
            't_water':        145,
            #flankers
            'f_baby' :        155,
            'f_bell':         156,
            'f_bird':         157,
            'f_burp':         158,
            'f_dog':          159,
            'f_drum':         160,
            'f_knock':        161,
            'f_laugh':        162,
            'f_phone':        163,
            'f_train':        164,
            'f_water':        165,
}
# event codes 1 - 24 represent flanker and target stim codes. They are NOT explicity defined in dissertation.
for i in range(1,13):
    event_id["flanker_stim_{0}".format(i)] = i
    
for j in range(13, 25):
    event_id["target_stim_{0}".format(j)] = j

In [ ]:
epoch = mne.Epochs(raw, events=events, event_id = event_id, preload = True, )

In [ ]:
epoch.plot()

In [ ]:
print('Original sampling rate:', epoch.info['sfreq'], 'Hz')

In [ ]:
epoch_resampled = epoch.copy().resample(100, npad='auto')
print('New sampling rate:', epoch_resampled.info['sfreq'], 'Hz')

In [ ]:
epoch.plot(n_channels=20, n_epochs=1,title="events")

In [ ]:
X=epoch.get_data();
y=epoch.events[:, 2] 

In [ ]:
print (x)

In [ ]:
epochs = epoch[(epoch.events[:,2] == 5) | (epoch.events[:,2] == 6)]
y=epochs.events[:, 2] 
X=epochs.get_data();

print(y)
print(X)

In [ ]:
y=epochs.events[:, 2] 
X=epochs.get_data();
clf=make_pipeline(StandardScaler(), LinearRegression())

time_decod = SlidingEstimator(clf, n_jobs=1)

scores = cross_val_multiscore(time_decod, X, y, cv=8, n_jobs=1)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epoch.times, scores, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding for Subject 1 Run2')
plt.show()

In [ ]:

def classify_test_KNN(X,y):

    clf = make_pipeline(Vectorizer(),
                        #StandardScaler(),
                        KNN(n_neighbors=11))
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
    clf.fit(X_train, y_train)
    
    score = clf.score(X_test, y_test)
    
    return score

In [ ]:
clf = make_pipeline(Vectorizer(),StandardScaler(),
                        KNN(n_neighbors=13))
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
clf.fit(X_train, y_train)
    
score = clf.score(X_test, y_test)
print (score)

In [ ]:

def filter_events(epochs, filter_for):

    if len(filter_for) >= len(epochs.event_id):
        return None
    
    else:
        temp_epochs = []
        for i in range(len(filter_for)):
            temp_epochs.append(epochs[epochs.events[:,-1] == filter_for[i]])
        
        return mne.concatenate_epochs(temp_epochs)

In [ ]:
pca = PCA(n_components=4)
pca_result = pca.fit_transform(X)

In [ ]:
ise= mne.decoding.UnsupervisedSpatialFilter(NMF(),average=False)
ise_data= ise.fit_transform(X)